In [1]:
import os
import pprint
from dotenv import load_dotenv
from typing import List, Optional
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import Qdrant
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field
from qdrant_client import QdrantClient
from langchain_ollama import OllamaLLM
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import torch
from langchain.embeddings.base import Embeddings

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98;} </style>"))

In [3]:
# Cargar variables de entorno 
load_dotenv()
print("Variables de entorno cargadas")

Variables de entorno cargadas


In [4]:
# LLM
llm = OllamaLLM(model="hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:Q8_0")
print("LLM inicializado")
# ollama run hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:Q8_0

LLM inicializado


__Modelo de embedding  intfloat/multilingual-e5-large-instruct__
https://huggingface.co/intfloat/multilingual-e5-large-instruct

In [ ]:

# Wrapper custom para E5 con soporte GPU
class E5InstructEmbeddings(Embeddings):
    def __init__(self, model_name="intfloat/multilingual-e5-large-instruct", device=None):
        from sentence_transformers import SentenceTransformer
        
        # Detectar dispositivo si no se especifica
        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)
        print(f'Modelo E5 cargado en dispositivo: {device}')
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Para documentos - añadir prefix 'passage:'"""
        prefixed_texts = [f"passage: {text}" for text in texts]
        return self.model.encode(prefixed_texts, device=self.device).tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Para queries - añadir prefix 'query:'"""
        prefixed_text = f"query: {text}"
        return self.model.encode([prefixed_text], device=self.device)[0].tolist()

# Especificamos que use la GPU (cuda) si está disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando dispositivo para embeddings: {device}')

# Usar el wrapper custom con GPU support
embeddings = E5InstructEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    device=device
)

# Configuración de la colección (sin cambios)
COLLECTION_NAME = "idearq-e5-large-instruct"
VECTOR_SIZE = 1024

# Inicializar cliente de Qdrant
client = QdrantClient(url="http://localhost:6333")

# Borrar y recrear la colección
if COLLECTION_NAME in [c.name for c in client.get_collections().collections]:
    client.delete_collection(COLLECTION_NAME)

client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE),
)

print(f"Colección '{COLLECTION_NAME}' creada/recreada exitosamente.")

Usando dispositivo para embeddings: cuda
Modelo E5 cargado en dispositivo: cuda


C:\Users\elevi\AppData\Local\Temp\ipykernel_25700\3741368808.py:49: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Colección 'idearq-e5-large-instruct' creada/recreada exitosamente.


In [8]:
client = QdrantClient(url="http://localhost:6333")
info = client.get_collection(COLLECTION_NAME)
assert info.config.params.vectors.size == 1024, "La colección no está configurada para vectores de 1024 dimensiones."
print("¡La dimensión de los vectores (1024) es correcta!")

¡La dimensión de los vectores (1024) es correcta!


In [9]:
import logging
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

all_docs = []
folder_path = Path("pdf_articulos_idearq/") 

logging.info(f"Iniciando la carga de documentos desde: {folder_path.resolve()}")

if not folder_path.is_dir():
    logging.error(f"El directorio especificado no existe: {folder_path.resolve()}")
else:
    # Usamos .glob() que es eficiente para buscar archivos
    for pdf_file in folder_path.glob("*.pdf"):
        try:
            loader = PyMuPDFLoader(str(pdf_file), mode="single")
            docs_from_file = loader.load()
            all_docs.extend(docs_from_file)
            # logging.debug(f"Cargado: {pdf_file.name}") 
        except Exception as e:
            # Este bloque capturará si un PDF específico está dañado y no se puede leer
            logging.warning(f"No se pudo cargar el archivo '{pdf_file.name}'. Saltando. Error: {e}")

    logging.info(f"Carga finalizada. Se han cargado un total de {len(all_docs)} documentos.")

2025-08-03 17:43:18,804 - INFO - Iniciando la carga de documentos desde: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq
2025-08-03 17:44:24,173 - INFO - Carga finalizada. Se han cargado un total de 300 documentos.


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import logging

# Configurar un logger para controlar el proceso
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

text_splitter_config = {
    "chunk_size": 1000,
    "chunk_overlap": 200, # 15-20% de chunk_size
    "length_function": len,
    "add_start_index": True,
    "separators": ["\n\n", "\n", ". ", " ", ""],
}

# El operador ** desempaqueta el diccionario como argumentos 
text_splitter = RecursiveCharacterTextSplitter(**text_splitter_config)

all_splits = text_splitter.split_documents(all_docs)

logging.info(f"Se dividieron {len(all_docs)} documentos en un total de {len(all_splits)} chunks.")

2025-08-03 17:44:28,097 - INFO - Se dividieron 300 documentos en un total de 50687 chunks.


In [11]:
from langchain_qdrant import Qdrant

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "idearq-e5-large-instruct"

# Configura el logger si no lo has hecho en una celda anterior.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 3. CREAR EL OBJETO 'vector_store' 
# Primero, nos conectamos al cliente de Qdrant 
client = QdrantClient(url=QDRANT_URL)

# Ahora, creamos la variable 'vector_store', que es el objeto de LangChain
# Este objeto "envuelve" al cliente y sabe cómo usar la función de embeddings
logging.info(f"Conectando a la colección '{COLLECTION_NAME}' a través de LangChain...")
vector_store = Qdrant(
    client=client,
    collection_name=COLLECTION_NAME,
    embeddings=embeddings,
)
logging.info("Objeto 'vector_store' listo para la ingesta")

# 4. INGESTA DE DOCUMENTOS 
logging.info(f"Iniciando la ingesta de {len(all_splits)} chunks...")

vector_store.add_documents(all_splits)

logging.info("Ingesta completada")


2025-08-03 17:48:37,952 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-03 17:48:37,956 - INFO - Conectando a la colección 'idearq-e5-large-instruct' a través de LangChain...
C:\Users\elevi\AppData\Local\Temp\ipykernel_25700\4001779736.py:16: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  vector_store = Qdrant(
2025-08-03 17:48:37,958 - INFO - Objeto 'vector_store' listo para la ingesta
2025-08-03 17:48:37,959 - INFO - Iniciando la ingesta de 50687 chunks...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-03 17:48:46,805 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-03 17:48:54,336 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


...

2025-08-03 19:55:22,905 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-03 19:55:37,796 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-03 19:55:44,264 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"
2025-08-03 19:55:44,397 - INFO - Ingesta completada


In [12]:
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")

COLLECTION_NAME = "idearq-e5-large-instruct" 

print("\n--- Verificación en Qdrant ---")
print(f"Información de la colección '{COLLECTION_NAME}':")

# Usamos client.count() para obtener el número exacto de vectores
try:
    count_result = client.count(
        collection_name=COLLECTION_NAME, 
        exact=True
    )
    
    total_vectors = count_result.count
    
    if 'all_splits' in locals():
      print(f" - Chunks procesados en esta ejecución: {len(all_splits)}")

    print(f" - Número TOTAL de vectores en la colección: {total_vectors}")

except Exception as e:
    print(f"No se pudo obtener el conteo de la colección '{COLLECTION_NAME}'. Error: {e}")

2025-08-03 19:55:44,581 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-03 19:55:44,646 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/count "HTTP/1.1 200 OK"



--- Verificación en Qdrant ---
Información de la colección 'idearq-e5-large-instruct':
 - Chunks procesados en esta ejecución: 50687
 - Número TOTAL de vectores en la colección: 50687


In [13]:
from langchain_core.prompts import ChatPromptTemplate


prompt_template = """
Eres un asistente experto en Prehistoria y Arqueología. Tu tarea es responder a la pregunta del usuario de forma clara y concisa, utilizando únicamente la información proporcionada en el siguiente contexto.

Contexto:
{context}

Pregunta: {query}

Instrucciones:
1. Lee la pregunta y el contexto cuidadosamente.
2. Sintetiza la información de los diferentes fragmentos del contexto para formular una respuesta completa.
3. Si el contexto contiene fechas o cronologías relevantes para la pregunta, asegúrate de incluirlas en tu respuesta.
4. Si el contexto no contiene información suficiente para responder a la pregunta, responde honestamente que no puedes encontrar la respuesta en la información proporcionada. No inventes información.

Respuesta:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


In [17]:
# Estado del grafo con Pydantic
class State(BaseModel):
    question: str
    context: List[Document] = Field(default_factory=list)
    answer: Optional[str] = None

# Nodo retrieve
def retrieve(state: State) -> dict:
    retrieved_docs = vector_store.similarity_search(state.question, k = 4)
    return {"context": retrieved_docs}

# Nodo generate
def generate(state: State) -> dict:
    docs_content = "\n\n".join(doc.page_content for doc in state.context)
    messages = prompt.invoke({"query": state.question, "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [18]:
# Test del grafo completo
result = graph.invoke({
    "question": "¿Cuál es la secuencia cultural de la Prehistoria y Protohistoria de Portugal?"
})

print(f"Pregunta: {result['question']}")
print(f"Documentos encontrados: {len(result['context'])}")
print(f"Respuesta: {result['answer']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-03 20:02:39,097 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/search "HTTP/1.1 200 OK"
2025-08-03 20:03:33,947 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Pregunta: ¿Cuál es la secuencia cultural de la Prehistoria y Protohistoria de Portugal?
Documentos encontrados: 4
Respuesta: Lo siento, pero no puedo proporcionar una respuesta detallada sobre la secuencia cultural de la Prehistoria y Protohistoria de Portugal basándome solo en el contexto proporcionado. El contexto incluye varios artículos académicos relacionados con la prehistoria y arqueología de diferentes regiones de Europa, pero no cubre específicamente la secuencia cultural de Portugal.

Sin embargo, puedo intentar ofrecer una respuesta general basada en la información disponible:

En Portugal, como en muchos otros países europeos, la transición del Paleolítico Superior al Neolítico es un período significativo en la historia prehistórica. Aunque no hay una secuencia cultural única para Portugal que se pueda resumir en una sola respuesta, podemos identificar algunos puntos clave basados en la información disponible.

La transición del Paleolítico Superior al Neolítico en Portugal

In [19]:
result = graph.invoke({"question": "¿Cuáles son las dataciones más antiguas para el Neolítico Peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-03 20:04:42,572 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/search "HTTP/1.1 200 OK"
2025-08-03 20:04:48,286 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2018-01-23T12:29:09+01:00', 'source': 'pdf_articulos_idearq\\998_maa_2017.pdf', 'file_path': 'pdf_articulos_idearq\\998_maa_2017.pdf', 'total_pages': 24, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-01-23T12:29:12+01:00', 'trapped': '', 'modDate': "D:20180123122912+01'00'", 'creationDate': "D:20180123122909+01'00'", 'start_index': 72214, '_id': 'ea4c05bb-fcdc-426e-9a39-8d7a328e364f', '_collection_name': 'idearq-e5-large-instruct'}, page_content='neo central peninsular. In: Bernabéu, J., Rojo, M.A., Molina, Ll. \n(Coords.), Las primeras producciones cerámicas: el VI milenio \ncal AC en la Península Ibérica, 153-178, Universitat de Valèn-\ncia Saguntum PLAV Extra-12. \nBernabéu, J., Martí, B., 2014. The first agricultural groups in the \nIberian Peninsula. In: Manen, C., Perrin, T., Guilaine, J. (Eds.), \nL

In [20]:
result = graph.invoke({"question": "¿Hay evidencias de caballo doméstico en el Calcolítico peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-03 20:08:36,071 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/search "HTTP/1.1 200 OK"
2025-08-03 20:08:41,118 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2017-05-25T17:11:56+08:00', 'source': 'pdf_articulos_idearq\\999_navarrete_2017.pdf', 'file_path': 'pdf_articulos_idearq\\999_navarrete_2017.pdf', 'total_pages': 26, 'format': 'PDF 1.5', 'title': 'Feeding management strategies among the early Neolithic pigs in the NE of the Iberian Peninsula', 'author': 'V. Navarrete, A. C. Colonese, C. Tornero, F. Antolín, M. Von Tersch, M. Eulàlia Subirà, P. Comes, A. Rosell-Melé, M. Saña', 'subject': '', 'keywords': '10.1002/oa.2598 and Iberian Peninsula; Early Neolithic; husbandry practices; pig diet; stable carbon and nitrogen isotopes', 'moddate': '2017-05-25T17:56:53+08:00', 'trapped': '', 'modDate': "D:20170525175653+08'00'", 'creationDate': "D:20170525171156+08'00'", 'start_index': 37443, '_id': '741d48a3-9062-4629-8587-6141d43d937b', '_collection_name': 'idearq-e5-large-instruct'}, page_content='husbandry in Sardi

In [21]:
result = graph.invoke({"question": "¿Cuál es la secuencia cultural de la Prehistoria y Protohistoria de Portugal?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-03 20:09:14,122 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/search "HTTP/1.1 200 OK"
2025-08-03 20:09:20,640 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2018-01-23T12:29:09+01:00', 'source': 'pdf_articulos_idearq\\998_maa_2017.pdf', 'file_path': 'pdf_articulos_idearq\\998_maa_2017.pdf', 'total_pages': 24, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-01-23T12:29:12+01:00', 'trapped': '', 'modDate': "D:20180123122912+01'00'", 'creationDate': "D:20180123122909+01'00'", 'start_index': 79361, '_id': '03c896e9-330f-437d-8668-bbba89b4c705', '_collection_name': 'idearq-e5-large-instruct'}, page_content='Carvalho, A.F., 2008. A Neolitização do Portugal Meridional. Os \nexemplos do Maciço Calcário Estremenho e do Algarve Oci-\ndental. Universidade do Algarve. Promontoria Monográfica 12. \nCarvalho, A.F., 2010. Chronology and geography of the Me-\nsolithic-Neolithic transition in Portugal. In: Henning, J., Leube, \nA., Biermann, F. (Eds.). On Pre- and Earlier Histor

In [22]:
result = graph.invoke({"question": "¿Cuándo se produce el poblamiento de Baleares?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-03 20:09:57,359 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/search "HTTP/1.1 200 OK"
2025-08-03 20:10:03,892 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2018-01-23T12:29:09+01:00', 'source': 'pdf_articulos_idearq\\998_maa_2017.pdf', 'file_path': 'pdf_articulos_idearq\\998_maa_2017.pdf', 'total_pages': 24, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-01-23T12:29:12+01:00', 'trapped': '', 'modDate': "D:20180123122912+01'00'", 'creationDate': "D:20180123122909+01'00'", 'start_index': 92122, '_id': '749973f5-fa6f-4ff9-a723-aa904db137b1', '_collection_name': 'idearq-e5-large-instruct'}, page_content='d’Estudis Penedesencs. Vilafranca del Penedès, 117-134. \nOms, F.X., Gibaja, J.F., Mazzuco, N., Guilaine, J., 2016b. Revi-\nsión radiocarbónica y cronocultural del Neolítico antiguo de la \nBalma Margineda (Aixovall, Andorra). Trabajos de Prehistoria \n73(1), 29-46.\nOms, F.X., Martín, A., Esteve, X., Mestres, J., Morell, B., Gibaja, \nJ.F., 2016c. The Neolithic i

In [ ]:
question = "¿Cuáles son las dataciones más antiguas para el Neolítico Peninsular?"

# Llamamos directamente al método de búsqueda, igual que en tu nodo 'retrieve'
retrieved_docs = vector_store.similarity_search(question)

print("--- Documentos recuperados directamente ---")
if not retrieved_docs:
    print("¡No se recuperó ningún documento! Este es probablemente el problema.")
else:
    for i, doc in enumerate(retrieved_docs):
        print(f"Documento {i+1}:")
        print(f"Contenido: {doc.page_content}")
        # También es útil ver los metadatos para saber de dónde viene el documento
        print(f"Metadatos: {doc.metadata}")
        print("-" * 20)



2025-08-02 13:38:04,742 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-e5-large-instruct/points/search "HTTP/1.1 200 OK"


--- Documentos recuperados directamente ---
Documento 1:
Contenido: neo central peninsular. In: Bernabéu, J., Rojo, M.A., Molina, Ll. 
(Coords.), Las primeras producciones cerámicas: el VI milenio 
cal AC en la Península Ibérica, 153-178, Universitat de Valèn-
cia Saguntum PLAV Extra-12. 
Bernabéu, J., Martí, B., 2014. The first agricultural groups in the 
Iberian Peninsula. In: Manen, C., Perrin, T., Guilaine, J. (Eds.), 
La transition néolithique en Mediterranée. The Neolithic transi-
tion in the Mediterranean, 419-438, Ed. Érrance- AEP. 
Bernabéu, J., Molina, Ll., 2009. La Cova de les Cendres (Mo-
raira-Teulada, Alicante). MARQ. Diputació d’Alacant. Serie Ma-
yor, 6.
Bernabéu, J., Molina, Ll., Esquembre, M.A., Ortega, J.R., Boro-
nat, J., 2009. La cerámica impresa mediterránea en el origen 
del Neolítico de la Península Ibérica? In: De Méditerranéen et 
d’ailleurs… Mèlanges offerts à Jean Guilaine, 83-95, Archives 
d’Ecologie Préhistorique, Toulouse. 
Bernabéu, J., Rojo, M.A., Molin

Ingesta de más embeddings

In [7]:
class E5InstructEmbeddings(Embeddings):
    def __init__(self, model_name="intfloat/multilingual-e5-large-instruct", device=None):
        from sentence_transformers import SentenceTransformer
        
        # Detectar dispositivo si no se especifica
        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)
        print(f'Modelo E5 cargado en dispositivo: {device}')
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Para documentos - añadir prefix 'passage:'"""
        prefixed_texts = [f"passage: {text}" for text in texts]
        return self.model.encode(prefixed_texts, device=self.device).tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Para queries - añadir prefix 'query:'"""
        prefixed_text = f"query: {text}"
        return self.model.encode([prefixed_text], device=self.device)[0].tolist()

# Especificamos que use la GPU (cuda) si está disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando dispositivo para embeddings: {device}')

# Usar el wrapper custom con GPU support
embeddings = E5InstructEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    device=device
)

Usando dispositivo para embeddings: cuda
Modelo E5 cargado en dispositivo: cuda


In [9]:
import logging
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

all_docs = []
folder_path = Path("pdf_articulos_idearq/pdf_articulos_idearq_2subida/") 

logging.info(f"Iniciando la carga de documentos desde: {folder_path.resolve()}")

if not folder_path.is_dir():
    logging.error(f"El directorio especificado no existe: {folder_path.resolve()}")
else:
    for pdf_file in folder_path.glob("*.pdf"):
        try:
            loader = PyMuPDFLoader(str(pdf_file), mode="single")
            docs_from_file = loader.load()
            all_docs.extend(docs_from_file)
            
        except Exception as e:
           
            logging.warning(f"No se pudo cargar el archivo '{pdf_file.name}'. Saltando. Error: {e}")

    logging.info(f"Carga finalizada. Se han cargado un total de {len(all_docs)} documentos.")

2025-08-12 11:04:59,522 - INFO - Iniciando la carga de documentos desde: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq\pdf_articulos_idearq_2subida
2025-08-12 11:05:31,722 - INFO - Carga finalizada. Se han cargado un total de 105 documentos.


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

text_splitter_config = {
    "chunk_size": 1000,
    "chunk_overlap": 200, # 15-20% de chunk_size
    "length_function": len,
    "add_start_index": True,
    "separators": ["\n\n", "\n", ". ", " ", ""],
}

text_splitter = RecursiveCharacterTextSplitter(**text_splitter_config)
all_splits = text_splitter.split_documents(all_docs)
logging.info(f"Se dividieron {len(all_docs)} documentos en un total de {len(all_splits)} chunks.")

2025-08-12 11:05:57,748 - INFO - Se dividieron 105 documentos en un total de 27103 chunks.


In [11]:
from langchain_qdrant import Qdrant

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "idearq-e5-large-instruct"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
client = QdrantClient(url=QDRANT_URL)
logging.info(f"Conectando a la colección '{COLLECTION_NAME}' a través de LangChain...")
vector_store = Qdrant(
    client=client,
    collection_name=COLLECTION_NAME,
    embeddings=embeddings,
)
logging.info("Objeto 'vector_store' listo para la ingesta")
logging.info(f"Iniciando la ingesta de {len(all_splits)} chunks...")
vector_store.add_documents(all_splits)
logging.info("Ingesta completada")

2025-08-12 11:05:58,225 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-12 11:05:58,228 - INFO - Conectando a la colección 'idearq-e5-large-instruct' a través de LangChain...
C:\Users\elevi\AppData\Local\Temp\ipykernel_3808\3589522177.py:9: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  vector_store = Qdrant(
2025-08-12 11:05:58,230 - INFO - Objeto 'vector_store' listo para la ingesta
2025-08-12 11:05:58,231 - INFO - Iniciando la ingesta de 27103 chunks...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

e:\ANACONDA\envs\rag\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
2025-08-12 11:06:08,574 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-12 11:06:16,261 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


...

2025-08-12 12:22:30,356 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-12 12:22:38,479 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-12 12:22:51,349 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-e5-large-instruct/points?wait=true "HTTP/1.1 200 OK"
2025-08-12 12:22:51,354 - INFO - Ingesta completada


In [12]:
from langchain_core.prompts import ChatPromptTemplate


prompt_template = """
Eres un asistente experto en Prehistoria y Arqueología. Tu tarea es responder a la pregunta del usuario de forma clara y concisa, utilizando únicamente la información proporcionada en el siguiente contexto.

Contexto:
{context}

Pregunta: {query}

Instrucciones:
1. Lee la pregunta y el contexto cuidadosamente.
2. Sintetiza la información de los diferentes fragmentos del contexto para formular una respuesta completa.
3. Si el contexto contiene fechas o cronologías relevantes para la pregunta, asegúrate de incluirlas en tu respuesta.
4. Si el contexto no contiene información suficiente para responder a la pregunta, responde honestamente que no puedes encontrar la respuesta en la información proporcionada. No inventes información.

Respuesta:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

Segunda tanda de preguntas

In [ ]:
result = graph.invoke({
    "question": "¿Cuál es la utilidad de los análisis de isótopos de estroncio en Arqueología?"
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "¿Cuáles son las características de la distribución geográfica de la muestra disponible de análisis de isótopos de estroncio en la Península Ibérica?"
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "¿Cuáles son las dataciones más antiguas para la minería del sílex en el centro peninsular?"
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "¿Cuáles son los principales modelos teóricos sobre la expansión del Neolítico en Europa?"
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Cementerios Mesolíticos con dataciones radiocarbónicas en la Península Ibérica."
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Principales yacimientos de la Segunda Edad del Hierro en la provincia de León."
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Cementerios Mesolíticos con dataciones radiocarbónicas en la Península Ibérica."
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Periodización del Bronce Final en el Levante de la Península Ibérica, cronología de las fases y principales ejemplos de yacimientos asignados a las mismas."
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Yacimientos Calcolíticos de la Península Ibérica  en los que se han hallado objetos de marfil."
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Cronología y districubión espacial del poblamiento neolítixco en la Meseta Sur."
})

print(f"Pregunta: {result['question']}\n")
print("Artículos recuperados:")
for i, doc in enumerate(result['context']):
    meta = doc.metadata
    titulo = meta.get('title', 'Sin título')
    cita = meta.get('citation', 'Sin cita')
    autores = meta.get('authors', 'Sin autores')
    año = meta.get('year', 'Sin año')
    print(f"\n--- Artículo {i+1} ---")
    print(f"Título: {titulo}")
    print(f"Cita: {cita}")
    print(f"Autores: {autores}")
    print(f"Año: {año}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "Dataciones más antiguas para el megalitismo en la zona Sureste de la Península Ibérica."
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "¿Cuáles son las más antiguas evidencias de perro doméstico en Europa?"
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")

In [ ]:
result = graph.invoke({
    "question": "¿En qué año excavaron en el yacimiento de La Bastida de Totana los hermanos Siret?"
})

print(f"Pregunta: {result['question']}")
print(f"\nContexto: {result['context']}")
print(f"\nRespuesta: {result['answer']}")